In [3]:
# For Analysis
import numpy as np
import pandas as pd

# For Visualizations
import matplotlib.pyplot as plt
import seaborn as sns

# For Calculations
from math import floor

#For Modeling
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression , Lasso, Ridge
from sklearn.ensemble import RandomForestRegressor

# For Validation
from sklearn.metrics import mean_squared_error, accuracy_score

# For Storing Models
import pickle
%matplotlib inline

# For Warnings
import warnings
warnings.filterwarnings("ignore")

In [4]:
listings = pd.read_csv('listings_2.csv')

In [5]:
listings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3818 entries, 0 to 3817
Data columns (total 60 columns):
id                                  3818 non-null int64
name                                3818 non-null object
summary                             3818 non-null object
space                               3818 non-null object
description                         3818 non-null object
neighborhood_overview               3818 non-null object
transit                             3818 non-null object
host_id                             3818 non-null int64
host_since                          3818 non-null object
host_response_time                  3818 non-null object
host_response_rate                  3818 non-null object
host_acceptance_rate                3818 non-null object
host_is_superhost                   3818 non-null object
host_neighbourhood                  3818 non-null object
host_listings_count                 3818 non-null float64
host_total_listings_count           381

In [6]:
listings.describe()

,id,host_id,host_listings_count,host_total_listings_count,accommodates,bathrooms,bedrooms,beds,price,guests_included,...,number_of_reviews,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,calculated_host_listings_count,reviews_per_month
count,3.818000e+03,3.818000e+03,3818.000000,3818.000000,3818.000000,3818.000000,3818.000000,3818.000000,3818.000000,3818.000000,...,3818.000000,3818.000000,3818.000000,3818.000000,3818.000000,3818.000000,3818.000000,3818.000000,3818.000000,3818.000000
mean,5.550111e+06,1.578556e+07,7.157412,7.157412,3.349398,1.258643,1.308015,1.735726,127.976166,1.672603,...,22.223415,94.584075,9.636721,9.566003,9.790990,9.815610,9.614196,9.457569,2.946307,2.065888
std,2.962660e+06,1.458382e+07,28.620929,28.620929,1.977599,0.589365,0.883252,1.139514,90.250022,1.311040,...,37.730892,6.508363,0.700898,0.781946,0.582518,0.548938,0.624517,0.741315,5.893029,1.814807
min,3.335000e+03,4.193000e+03,1.000000,1.000000,1.000000,0.000000,0.000000,1.000000,20.000000,0.000000,...,0.000000,20.000000,2.000000,3.000000,2.000000,2.000000,4.000000,2.000000,1.000000,0.020000
25%,3.258256e+06,3.275204e+06,1.000000,1.000000,2.000000,1.000000,1.000000,1.000000,75.000000,1.000000,...,2.000000,93.000000,9.000000,9.000000,10.000000,10.000000,9.000000,9.000000,1.000000,0.700000
50%,6.118244e+06,1.055814e+07,1.000000,1.000000,3.000000,1.000000,1.000000,1.000000,100.000000,1.000000,...,9.000000,96.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,1.000000,1.510000
75%,8.035127e+06,2.590309e+07,3.000000,3.000000,4.000000,1.000000,2.000000,2.000000,150.000000,2.000000,...,26.000000,100.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,2.000000,2.987500
max,1.034016e+07,5.320861e+07,502.000000,502.000000,16.000000,8.000000,7.000000,15.000000,1000.000000,15.000000,...,474.000000,100.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,37.000000,12.150000


In [7]:
#observe there are several comlumns with blanks
#Blank missing values in Amenities column
#It means that we filled missing values with "", so in the next few steps, we can took them as False(False means there is no amenities)
listings.loc[listings['amenities'] == '{}','amenities'] = ""

In [8]:
listings['amenities'].head()

0    {TV,"Cable TV",Internet,"Wireless Internet","A...
1    {TV,Internet,"Wireless Internet",Kitchen,"Free...
2    {TV,"Cable TV",Internet,"Wireless Internet","A...
3    {Internet,"Wireless Internet",Kitchen,"Indoor ...
4    {TV,"Cable TV",Internet,"Wireless Internet",Ki...
Name: amenities, dtype: object

In [9]:
#Remove comma and citation symbols and split the amenities with \\\as separator 
listings['amenities'] = listings['amenities'].map(
    lambda amns: "|||".join([amn.replace("}", "").replace("{", "").replace('"', "")\
                           for amn in amns.split(",")]))

In [10]:
listings['amenities'].head()

0    TV|||Cable TV|||Internet|||Wireless Internet||...
1    TV|||Internet|||Wireless Internet|||Kitchen|||...
2    TV|||Cable TV|||Internet|||Wireless Internet||...
3    Internet|||Wireless Internet|||Kitchen|||Indoo...
4    TV|||Cable TV|||Internet|||Wireless Internet||...
Name: amenities, dtype: object

In [11]:
#Take the unique list of amenities across all listings. Express it as an array for future use
#if the amentity appears, it will be in the quotation symbol
amenities = np.unique(np.concatenate(listings['amenities'].map(lambda amns: amns.split("|||")).values))
amenities

array(['', '24-Hour Check-in', 'Air Conditioning', 'Breakfast',
       'Buzzer/Wireless Intercom', 'Cable TV', 'Carbon Monoxide Detector',
       'Cat(s)', 'Dog(s)', 'Doorman', 'Dryer', 'Elevator in Building',
       'Essentials', 'Family/Kid Friendly', 'Fire Extinguisher',
       'First Aid Kit', 'Free Parking on Premises', 'Gym', 'Hair Dryer',
       'Hangers', 'Heating', 'Hot Tub', 'Indoor Fireplace', 'Internet',
       'Iron', 'Kitchen', 'Laptop Friendly Workspace',
       'Lock on Bedroom Door', 'Other pet(s)', 'Pets Allowed',
       'Pets live on this property', 'Pool', 'Safety Card', 'Shampoo',
       'Smoke Detector', 'Smoking Allowed', 'Suitable for Events', 'TV',
       'Washer', 'Washer / Dryer', 'Wheelchair Accessible',
       'Wireless Internet'], dtype='<U26')

In [14]:
#Map the presence or absence of amenities for each listing
#True for having this amentit y, False for not having this amentity
#Therefore, for those rows without values, all should be False, because they don't have any one of these amenities.
amenities_matrix = np.array([listings['amenities'].map(lambda amns: amn in amns).values for amn in amenities])
amenities_matrix

array([[ True,  True,  True, ...,  True,  True,  True],
       [False, False, False, ..., False, False, False],
       [ True, False,  True, ..., False, False, False],
       ...,
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [ True,  True,  True, ...,  True,  True,  True]])

In [15]:
#Make the amenities dataframe with boolean values
#True for having this amentity, False for not having this amentity
amen=pd.DataFrame(data=amenities_matrix.T, columns=amenities)
amen.head()

,,24-Hour Check-in,Air Conditioning,Breakfast,Buzzer/Wireless Intercom,Cable TV,Carbon Monoxide Detector,Cat(s),Dog(s),Doorman,...,Safety Card,Shampoo,Smoke Detector,Smoking Allowed,Suitable for Events,TV,Washer,Washer / Dryer,Wheelchair Accessible,Wireless Internet
0,True,False,True,False,False,True,False,False,False,False,...,False,False,False,False,False,True,True,False,False,True
1,True,False,False,False,True,False,True,False,False,False,...,True,False,True,False,False,True,True,False,False,True
2,True,False,True,False,False,True,True,True,True,False,...,False,True,True,False,False,True,True,False,False,True
3,True,False,False,False,False,False,True,False,False,False,...,True,True,True,False,False,False,True,False,False,True
4,True,False,False,False,False,True,True,False,False,False,...,False,True,True,False,False,True,False,False,False,True


In [16]:
#Concat the listing id to amen dataframe for further concatation
listings_amenities = pd.concat([listings['id'],amen], axis=1)
listings_amenities.head()

,id,,24-Hour Check-in,Air Conditioning,Breakfast,Buzzer/Wireless Intercom,Cable TV,Carbon Monoxide Detector,Cat(s),Dog(s),...,Safety Card,Shampoo,Smoke Detector,Smoking Allowed,Suitable for Events,TV,Washer,Washer / Dryer,Wheelchair Accessible,Wireless Internet
0,241032,True,False,True,False,False,True,False,False,False,...,False,False,False,False,False,True,True,False,False,True
1,953595,True,False,False,False,True,False,True,False,False,...,True,False,True,False,False,True,True,False,False,True
2,3308979,True,False,True,False,False,True,True,True,True,...,False,True,True,False,False,True,True,False,False,True
3,7421966,True,False,False,False,False,False,True,False,False,...,True,True,True,False,False,False,True,False,False,True
4,278830,True,False,False,False,False,True,True,False,False,...,False,True,True,False,False,True,False,False,False,True


In [17]:
#Remove second column from listings_amenities whose name is ""
listings_amenities=listings_amenities.drop("",axis=1)
listings_amenities.head()

,id,24-Hour Check-in,Air Conditioning,Breakfast,Buzzer/Wireless Intercom,Cable TV,Carbon Monoxide Detector,Cat(s),Dog(s),Doorman,...,Safety Card,Shampoo,Smoke Detector,Smoking Allowed,Suitable for Events,TV,Washer,Washer / Dryer,Wheelchair Accessible,Wireless Internet
0,241032,False,True,False,False,True,False,False,False,False,...,False,False,False,False,False,True,True,False,False,True
1,953595,False,False,False,True,False,True,False,False,False,...,True,False,True,False,False,True,True,False,False,True
2,3308979,False,True,False,False,True,True,True,True,False,...,False,True,True,False,False,True,True,False,False,True
3,7421966,False,False,False,False,False,True,False,False,False,...,True,True,True,False,False,False,True,False,False,True
4,278830,False,False,False,False,True,True,False,False,False,...,False,True,True,False,False,True,False,False,False,True


In [18]:
#Group some amentities with similar features and name them as a new column.
amenity_recode={
                'Air Conditioning':'Weather Control',
                'Indoor Fireplace':'Weather Control',
                'Heating':'Weather Control',
        
                'Carbon Monoxide Detector':'Safety Features',
                'Fire Extinguisher':'Safety Features',
                'First Aid Kit':'Safety Features',
                'Smoke Detector':'Safety Features',
                
                'Buzzer/Wireless Intercom':'Security Features',
                'Doorman':'Security Features',
                'Safety Card':'Security Features',
                'Lock on Bedroom Door':'Security Features',
                
                'Cat(s)':'Pet Friendly',
                'Dog(s)':'Pet Friendly',
                'Pets Allowed':'Pet Friendly',
                'Pets live on this property':'Pet Friendly',
                'Other pet(s)':'Pet Friendly',
                
                'Elevator in Building':'Access Friendly',
                'Wheelchair Accessible':'Access Friendly',
                
                'Essentials':'Essentials',
                'Hair Dryer':'Essentials',
                'Hangers':'Essentials',
                'Iron':'Essentials',
                'Shampoo':'Essentials',             
                
                'Cable TV':'TV',
                'TV':'TV',
                
                'Internet':'Internet',
                'Wireless Internet':'Internet',
                'Laptop Friendly Workspace':'Internet',
                
                'Dryer':'Laundry Facility',
                'Washer':'Laundry Facility',
                'Washer / Dryer':'Laundry Facility',
    
                #Leaving amenities as such which cannot be grouped
                #'Kitchen',
                #'Family/Kid Friendly', 
                #'Free Parking on Premises',
                #'Breakfast',
                #'24-Hour Check-in',
                #'Hot Tub',
                #'Pool',
                #'Gym',
                #'Smoking Allowed',
                #'Suitable for Events'
                #
}

In [19]:
#Melt the amenities dataframe and recode from the dictionary
listings_amenities_melt = listings_amenities.melt(id_vars=['id'], var_name='amenity')

#Recoding and putting in new column called amenity_modified
listings_amenities_melt = listings_amenities_melt.assign(
    amenity_modified = listings_amenities_melt.amenity.replace(amenity_recode)
)

listings_amenities_melt.head()

,id,amenity,value,amenity_modified
0,241032,24-Hour Check-in,False,24-Hour Check-in
1,953595,24-Hour Check-in,False,24-Hour Check-in
2,3308979,24-Hour Check-in,False,24-Hour Check-in
3,7421966,24-Hour Check-in,False,24-Hour Check-in
4,278830,24-Hour Check-in,False,24-Hour Check-in


In [20]:
#Pivot the melted dataframe before merging with original dataframe
listings_amenities_pivot = listings_amenities_melt.pivot_table(
    index='id',
    columns='amenity_modified',
    values='value', 
    aggfunc='max'
)

listings_amenities_pivot.head()

amenity_modified,24-Hour Check-in,Access Friendly,Breakfast,Essentials,Family/Kid Friendly,Free Parking on Premises,Gym,Hot Tub,Internet,Kitchen,Laundry Facility,Pet Friendly,Pool,Safety Features,Security Features,Smoking Allowed,Suitable for Events,TV,Weather Control
id,,,,,,,,,,,,,,,,,,,
3335,False,False,False,False,True,True,False,False,True,True,True,True,False,True,False,False,False,False,True
4291,False,False,False,True,False,True,False,False,True,False,True,False,False,True,False,False,False,True,True
5682,True,False,False,True,False,False,False,False,True,True,True,False,False,True,False,False,False,True,True
6606,False,False,False,True,False,True,False,True,True,True,False,True,False,False,False,False,False,True,True
7369,False,True,False,True,False,False,False,False,True,True,True,False,False,True,False,False,False,True,True


In [21]:
#Join the amenities dataframe back to the original listings dataframe
listings_joined=listings.join(listings_amenities_pivot,on="id",how="inner")
listings_joined.head()

,id,name,summary,space,description,neighborhood_overview,transit,host_id,host_since,host_response_time,...,Kitchen,Laundry Facility,Pet Friendly,Pool,Safety Features,Security Features,Smoking Allowed,Suitable for Events,TV,Weather Control
0,241032,Stylish Queen Anne Apartment,Chemically sensitive? We've removed the irrita...,Make your self at home in this charming one-be...,Make your self at home in this charming one-be...,"Queen Anne is a wonderful, truly functional vi...","Convenient bus stops are just down the block, ...",956883,11/8/2011,within a few hours,...,True,True,False,False,False,False,False,False,True,True
1,953595,Bright & Airy Queen Anne Apartment,Chemically sensitive? We've removed the irrita...,"Beautiful, hypoallergenic apartment in an extr...",Chemically sensitive? We've removed the irrita...,"Queen Anne is a wonderful, truly functional vi...","Convenient bus stops are just down the block, ...",5177328,21/2/2013,within an hour,...,True,True,False,False,True,True,False,False,True,True
2,3308979,New Modern House-Amazing water view,New modern house built in 2013. Spectacular s...,"Our house is modern, light and fresh with a wa...",New modern house built in 2013. Spectacular s...,Upper Queen Anne is a charming neighborhood fu...,A bus stop is just 2 blocks away. Easy bus a...,16708587,12/6/2014,within a few hours,...,True,True,True,False,True,False,False,False,True,True
3,7421966,Queen Anne Chateau,A charming apartment that sits atop Queen Anne...,"Our house is modern, light and fresh with a wa...",A charming apartment that sits atop Queen Anne...,Upper Queen Anne is a charming neighborhood fu...,A bus stop is just 2 blocks away. Easy bus a...,9851441,6/11/2013,within a few hours,...,True,True,False,False,True,True,False,False,False,True
4,278830,Charming craftsman 3 bdm house,Cozy family craftman house in beautiful neighb...,Cozy family craftman house in beautiful neighb...,Cozy family craftman house in beautiful neighb...,We are in the beautiful neighborhood of Queen ...,The nearest public transit bus (D Line) is 2 b...,1452570,29/11/2011,within an hour,...,True,False,False,False,True,False,False,False,True,True


In [22]:
#produce a new csv file.
listings_joined.to_csv('listings_3.csv',index=False)
